In [1]:
%load_ext autoreload
%autoreload 2
import sys 

sys.path.append('..')
from omegaconf import OmegaConf
from pprint import pprint
from dacite import from_dict
from dacite import Config as DaciteConfig
import torch

from xlstm.xlstm_lm_model import xLSTMLMModel, xLSTMLMModelConfig

In [2]:
# create new model to load the checkpoint into
xlstm_cfg = """ 
vocab_size: 50304
context_length: 516      
num_blocks: 4 #!
embedding_dim: 768 #!
tie_weights: false
weight_decay_on_embedding: false
mlstm_block:
  mlstm:
    conv1d_kernel_size: 2
    qkv_proj_blocksize: 2
    num_heads: 2
"""
cfg = OmegaConf.create(xlstm_cfg)
cfg = from_dict(data_class=xLSTMLMModelConfig, data=OmegaConf.to_container(cfg), config=DaciteConfig(strict=True))
model_new = xLSTMLMModel(cfg)

In [3]:
pprint(cfg)

xLSTMLMModelConfig(mlstm_block=mLSTMBlockConfig(mlstm=mLSTMLayerConfig(proj_factor=2.0,
                                                                       round_proj_up_dim_up=True,
                                                                       round_proj_up_to_multiple_of=64,
                                                                       _proj_up_dim=1536,
                                                                       conv1d_kernel_size=2,
                                                                       qkv_proj_blocksize=2,
                                                                       num_heads=2,
                                                                       embedding_dim=768,
                                                                       bias=False,
                                                                       dropout=0.0,
                                                                       context_length=516,
  

In [4]:
DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
x_in = torch.randint(0, 50304, (1, 32)).to(device=DEVICE)

In [6]:
model_new = model_new.to(device=DEVICE)

In [7]:
model_new

xLSTMLMModel(
  (xlstm_block_stack): xLSTMBlockStack(
    (blocks): ModuleList(
      (0-3): 4 x mLSTMBlock(
        (xlstm_norm): LayerNorm()
        (xlstm): mLSTMLayer(
          (proj_up): Linear(in_features=768, out_features=3072, bias=False)
          (q_proj): LinearHeadwiseExpand(in_features=1536, num_heads=768, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (k_proj): LinearHeadwiseExpand(in_features=1536, num_heads=768, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (v_proj): LinearHeadwiseExpand(in_features=1536, num_heads=768, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (conv1d): CausalConv1d(
            (conv): Conv1d(1536, 1536, kernel_size=(2,), stride=(1,), padding=(1,), groups=1536)
          )
          (conv_act_fn): SiLU()
          (mlstm_cell): mLSTMCell(
            (igate): Linear(in_features=4608, out_features=2, bias=True)
            (

In [8]:
y_new = model_new(x_in)

In [9]:
y_new.shape

torch.Size([1, 32, 50304])

In [10]:
y_new_step = []
state = None
for x in x_in.split(1, dim=1):
    y, state = model_new.step(x, state)
    y_new_step.append(y)
y_new_step = torch.cat(y_new_step, dim=1)
print(x.shape)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 2.95 GiB of which 7.25 MiB is free. Process 63280 has 141.38 MiB memory in use. Process 92007 has 83.38 MiB memory in use. Process 277123 has 91.38 MiB memory in use. Process 279082 has 995.38 MiB memory in use. Including non-PyTorch memory, this process has 1.65 GiB memory in use. Of the allocated memory 1.46 GiB is allocated by PyTorch, and 130.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
y_new_step.shape

torch.Size([1, 32, 50304])

In [12]:
y_new - y_new_step

tensor([[[ 5.9605e-08,  4.1723e-07,  5.9605e-08,  ...,  1.0431e-07,
           1.1921e-07,  3.7253e-07],
         [-2.3842e-07,  8.9407e-08,  5.9605e-08,  ..., -2.8312e-07,
           1.7881e-07,  3.5763e-07],
         [ 2.0862e-07, -2.9802e-08,  4.7684e-07,  ...,  3.5763e-07,
           3.5763e-07,  5.9605e-08],
         ...,
         [ 5.9605e-08,  5.9605e-07, -5.9605e-08,  ..., -8.9407e-08,
           5.9605e-08, -2.9802e-07],
         [ 2.3842e-07, -1.9372e-07, -1.3411e-07,  ..., -3.2783e-07,
           4.4703e-07, -5.9605e-08],
         [-1.1921e-07, -2.3842e-07,  6.2585e-07,  ..., -5.0664e-07,
          -1.7881e-07, -2.2352e-07]]], grad_fn=<SubBackward0>)

In [13]:
torch.allclose(y_new, y_new_step, atol=1e-5)

True

In [2]:
from omegaconf import OmegaConf
from dacite import from_dict
from dacite import Config as DaciteConfig
from xlstm import xLSTMLMModel, xLSTMLMModelConfig

xlstm_cfg = """ 
vocab_size: 50304
mlstm_block:
  mlstm:
    conv1d_kernel_size: 4
    qkv_proj_blocksize: 4
    num_heads: 4
slstm_block:
  slstm:
    backend: vanilla
    num_heads: 4
    conv1d_kernel_size: 4
    bias_init: powerlaw_blockdependent
  feedforward:
    proj_factor: 1.3
    act_fn: gelu
context_length: 256
num_blocks: 7
embedding_dim: 128
slstm_at: [1]
"""
cfg = OmegaConf.create(xlstm_cfg)
cfg = from_dict(data_class=xLSTMLMModelConfig, data=OmegaConf.to_container(cfg), config=DaciteConfig(strict=True))
xlstm_stack = xLSTMLMModel(cfg)

x = torch.randint(0, 50304, size=(4, 256)).to("cuda")
xlstm_stack = xlstm_stack.to("cuda")
y = xlstm_stack(x)
y.shape[1:] == (256, 50304)

True

In [3]:
import torch

from xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig,
)

cfg = xLSTMBlockStackConfig(
    mlstm_block=mLSTMBlockConfig(
        mlstm=mLSTMLayerConfig(
            conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4
        )
    ),
    slstm_block=sLSTMBlockConfig(
        slstm=sLSTMLayerConfig(
            backend="vanilla",
            num_heads=4,
            conv1d_kernel_size=4,
            bias_init="powerlaw_blockdependent",
        ),
        feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),
    ),
    context_length=256,
    num_blocks=7,
    embedding_dim=128,
    slstm_at=[1],

)

xlstm_stack = xLSTMBlockStack(cfg)

x = torch.randn(4, 256, 128).to("cuda")
xlstm_stack = xlstm_stack.to("cuda")
y = xlstm_stack(x)
y.shape == x.shape

True